In [4]:
# необходимые зависимости

import string

import numpy as np
import pandas as pd

In [2]:
A = np.array(
    [
        [np.nan, 20, 18, 12, 8],
        [5, np.nan, 14, 7, 11],
        [12, 18, np.nan, 6, 11],
        [11, 17, 11, np.nan, 12],
        [5, 5, 5, 5, np.nan]
    ],
    dtype=float
)

A

array([[nan, 20., 18., 12.,  8.],
       [ 5., nan, 14.,  7., 11.],
       [12., 18., nan,  6., 11.],
       [11., 17., 11., nan, 12.],
       [ 5.,  5.,  5.,  5., nan]])

In [3]:
B = np.array(
    [
        [np.nan, 68, 73, 24, 70, 9],
        [58, np.nan, 16, 44, 11, 92],
        [63, 9, np.nan, 86, 13, 18],
        [17, 34, 76, np.nan, 52, 70],
        [60, 18, 3, 45, np.nan, 58],
        [16, 82, 11, 60, 48, np.nan]
    ],
    dtype=float
)

B

array([[nan, 68., 73., 24., 70.,  9.],
       [58., nan, 16., 44., 11., 92.],
       [63.,  9., nan, 86., 13., 18.],
       [17., 34., 76., nan, 52., 70.],
       [60., 18.,  3., 45., nan, 58.],
       [16., 82., 11., 60., 48., nan]])

In [15]:
class Node:
   
    def __init__(self, frame: pd.DataFrame = None, weight: float = None):
        self.is_leaf = True
        self.left = None
        self.right = None
        self.frame = frame
        self.weight = weight
        
    def grow_left(self, frame: pd.DataFrame, weight: float) -> None:
        self.left = Node(frame, weight)
        self._check_growth()
        
    def grow_right(self, frame: pd.DataFrame, weight: float) -> None:
        self.right = Node(frame, weight)
        self._check_growth()
        
    def _check_growth(self) -> None:
        if self.left is not None and self.right is not None:
            self.is_leaf = False
            
    def find_least_leaf(self) -> Node:
        if self.is_leaf:
            return self
        elif self.left is not None and self.right is not None:
            left = self.left
            right = self.right

            if left.weight < right.weight:
                left.is_leaf = False
                return left
            else:
                right.is_leaf = False
                return right

In [19]:
class BranchAndBoundSolver:
    
    def __init__(self, initial_matrix: np.ndarray):
        
        labels = list(string.ascii_uppercase[:initial_matrix.shape[0]])
        
        self.initial_frame = pd.DataFrame(
            initial_matrix.copy(),
            columns=labels,
            index=labels
        )
        
        self.root = Node(
            *BranchAndBoundSolver.reduction(self.initial_frame)
        )
        
        
    def solve(self):
        
        for i in range(10):
            leaf = self.root.find_least_leaf()
            
            max_element_value, max_element_name = BranchAndBoundSolver.max_element_analysis(leaf.frame)
            
            leaf.grow_left(*BranchAndBoundSolver.remove_path(leaf.frame, max_element_name))
            
            leaf.grow_right(leaf.frame.copy(), leaf.weight + max_element_value)
            
    @staticmethod
    def remove_path(frame: pd.DataFrame, max_element_name):
        local_frame = frame.copy(deep=True)
        
        
            
    @staticmethod
    def max_element_analysis(frame: pd.DataFrame):
        local_matrix = frame.to_numpy().copy()
        new_matrix = np.zeros(shape=local_matrix.shape)

        for i in np.arange(local_matrix.shape[0]):
            for j in np.arange(local_matrix.shape[1]):
                stash = local_matrix[i, j]
                local_matrix[i, j] = np.nan

                axis0_min = np.nanmin(local_matrix[i])
                axis1_min = np.nanmin(local_matrix.T[j])

                new_matrix[i, j] = axis0_min + axis1_min

                local_matrix[i, j] = stash

        max_value = np.nanmax(new_matrix)

        max_value_position = np.array(
            np.where(new_matrix == max_value)
        ).T[0]

        return max_value, max_value_position
        
            
        
    @staticmethod
    def reduction(frame: pd.DataFrame):
        local_matrix = frame.to_numpy().copy()
        weight = 0

        axis1_mins = np.nanmin(local_matrix, axis=1).reshape(-1, 1)
        weight += axis1_mins.sum()
        local_matrix -= axis1_mins

        axis0_mins = np.nanmin(local_matrix, axis=0)
        weight += axis0_mins.sum()
        local_matrix -= axis0_mins
        
        new_frame = pd.DataFrame(local_matrix, columns=frame.columns, index=frame.index)
        
        return new_frame, weight

In [20]:
solver = BranchAndBoundSolver(A)

solver.solve()